# T1580 - Cloud Infrastructure Discovery
An adversary may attempt to discover infrastructure and resources that are available within an infrastructure-as-a-service (IaaS) environment. This includes compute service resources such as instances, virtual machines, and snapshots as well as resources of other services including the storage and database services.

Cloud providers offer methods such as APIs and commands issued through CLIs to serve information about infrastructure. For example, AWS provides a <code>DescribeInstances</code> API within the Amazon EC2 API that can return information about one or more instances within an account, the <code>ListBuckets</code> API that returns a list of all buckets owned by the authenticated sender of the request, the <code>HeadBucket</code> API to determine a bucket’s existence along with access permissions of the request sender, or the <code>GetPublicAccessBlock</code> API to retrieve access block configuration for a bucket.(Citation: Amazon Describe Instance)(Citation: Amazon Describe Instances API)(Citation: AWS Get Public Access Block)(Citation: AWS Head Bucket) Similarly, GCP's Cloud SDK CLI provides the <code>gcloud compute instances list</code> command to list all Google Compute Engine instances in a project (Citation: Google Compute Instances), and Azure's CLI command <code>az vm list</code> lists details of virtual machines.(Citation: Microsoft AZ CLI) In addition to API commands, adversaries can utilize open source tools to discover cloud storage infrastructure through [Wordlist Scanning](https://attack.mitre.org/techniques/T1595/003).(Citation: Malwarebytes OSINT Leaky Buckets - Hioureas)

An adversary may enumerate resources using a compromised user's access keys to determine which are available to that user.(Citation: Expel IO Evil in AWS) The discovery of these available resources may help adversaries determine their next steps in the Cloud environment, such as establishing Persistence.(Citation: Mandiant M-Trends 2020)An adversary may also use this information to change the configuration to make the bucket publicly accessible, allowing data to be accessed without authentication. Adversaries have also may use infrastructure discovery APIs such as <code>DescribeDBInstances</code> to determine size, owner, permissions, and network ACLs of database resources. (Citation: AWS Describe DB Instances) Adversaries can use this information to determine the potential value of databases and discover the requirements to access them. Unlike in [Cloud Service Discovery](https://attack.mitre.org/techniques/T1526), this technique focuses on the discovery of components of the provided services rather than the services themselves.

## Atomic Tests

### Atomic Test #1 - AWS - EC2 Enumeration from Cloud Instance
This atomic runs several API calls (sts:GetCallerIdentity, s3:ListBuckets, iam:GetAccountSummary, iam:ListRoles, iam:ListUsers, iam:GetAccountAuthorizationDetails, ec2:DescribeSnapshots, cloudtrail:DescribeTrails, guardduty:ListDetectors) from the context of an EC2 instance role. This simulates an attacker compromising an EC2 instance and running initial discovery commands on it. This atomic test leverages a tool called stratus-red-team built by DataDog (https://github.com/DataDog/stratus-red-team). Stratus Red Team is a self-contained binary. You can use it to easily detonate offensive attack techniques against a live cloud environment. Ref: https://stratus-red-team.cloud/attack-techniques/AWS/aws.discovery.ec2-enumerate-from-instance/

**Supported Platforms:** linux, macos, iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Stratus binary must be present at the (#{stratus_path}/stratus)

##### Check Prereq Commands:
```sh
if test -f "$PathToAtomicsFolder/T1580/src/stratus"; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```sh
if [ "$(uname)" = "Darwin" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep -i Darwin_x86_64 | cut -d '"' -f 4); wget -q -O $PathToAtomicsFolder/T1580/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1580/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1580/src/
elif [ "$(expr substr $(uname) 1 5)" = "Linux" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep -i linux_x86_64 | cut -d '"' -f 4); wget -q -O $PathToAtomicsFolder/T1580/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1580/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1580/src/
fi 

```
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```sh
echo "Please install the aws-cli and configure your AWS default profile using: aws configure"

```

In [ ]:
Invoke-AtomicTest T1580 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
export AWS_REGION=us-west-2
cd $PathToAtomicsFolder/T1580/src
echo "Stratus: Start Warmup."
./stratus warmup aws.discovery.ec2-enumerate-from-instance
echo "Stratus: Start Detonate."
./stratus detonate aws.discovery.ec2-enumerate-from-instance
```

In [ ]:
Invoke-AtomicTest T1580 -TestNumbers 1

#### Cleanup: 
```sh
cd $PathToAtomicsFolder/T1580/src
echo "Stratus: Start Cleanup."
./stratus cleanup aws.discovery.ec2-enumerate-from-instance
echo "Removing Stratus artifacts from local machine."
rm -rf stratus*
```

In [ ]:
Invoke-AtomicTest T1580 -TestNumbers 1 -Cleanup

## Detection
Establish centralized logging for the activity of cloud infrastructure components. Monitor logs for actions that could be taken to gather information about cloud infrastructure, including the use of discovery API calls by new or unexpected users and enumerations from unknown or malicious IP addresses. To reduce false positives, valid change management procedures could introduce a known identifier that is logged with the change (e.g., tag or header) if supported by the cloud provider, to help distinguish valid, expected actions from malicious ones.


## Shield Active Defense
### Decoy System 
 Configure a computing system to serve as an attack target or experimental environment.   

 A decoy system is a computing resource presented to the adversary in support of active defense.  The underlying system can be real, virtual, or simulated, and can be presented as one of a variety of IT devices including user workstations, servers, networking systems, IOT (embedded devices), mobile systems like phones, etc.
#### Opportunity
There is an opportunity to introduce decoy information, users, systems, etc. to influence an adversary's future actions.
#### Use Case
A defender can deploy a diverse set of decoy systems to impact an adversary's level of effort during recon activity.
#### Procedures
Use an isolated system to visit a suspected compromised website.  Collect any associated scripting code or files dropped onto the system.
Setup a server which appears to be something that is commonly expected within a network, such as web server.
